##### **** These pip install need to be adapted to use the appropriate release level. Currently used for testing alpha release of transforms 1.0

In [ ]:
%%capture
!pip install "data-prep-toolkit[ray]==0.2.3.post1"
!pip install "data-prep-toolkit-transforms[language]"
!pip install fasttext-wheel
import pyarrow.parquet as pq
import pandas as pd

Configure and run web2parquet

In [ ]:
%%capture
import urllib.request, os
os.mkdir("files-web2parquet", exist_ok=True)
urllib.request.urlretrieve("https://arxiv.org/pdf/2408.09869", 
                           "files-web2parquet/2408.09869.pdf")

In [ ]:
##### **** The specified downloads folder will include the downloaded file(s).
#import glob
#glob.glob("files-web2parquet/*")

Configure and run Pdf2Parquet

In [ ]:
from dpk_pdf2parquet.ray.transform import Pdf2Parquet
Pdf2Parquet(input_folder= "files-web2parquet",
               output_folder= "files-pdf2parquet",
               data_files_to_use=['.pdf'],
               run_locally= True,
               pdf2parquet_contents_type='text/markdown').transform()

In [ ]:
##### **** To explote the output from pdf2parquet, run the code below
#table = pq.read_table('files-pdf2parquet/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run DocChunk

In [ ]:
%%capture
from dpk_doc_chunk.ray.transform import DocChunk
DocChunk(input_folder='files-pdf2parquet',
        output_folder='files-doc-chunk',
        run_locally= True,
        doc_chunk_chunking_type= "li_markdown").transform()

In [ ]:
##### **** To explote the output from doc-chunk, run the code below
#table = pq.read_table('files-doc-chunk/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and Run Exact dedup

In [ ]:
from dpk_ededup.ray.transform import Ededup
Ededup(input_folder="files-doc-chunk",
    output_folder="files-ededup",
    ededup_doc_column="contents",
    run_locally= True,
    ededup_num_hashes= 2,
    ededup_doc_id_column="document_id").transform()

In [ ]:
##### **** To explote the output from eDedup, run the code below
#table = pq.read_table('files-ededup/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Lang Id

In [ ]:
from dpk_lang_id.ray.transform import LangId
LangId(input_folder= "files-ededup",
        output_folder= "files-langId",
        lang_id_model_credential= "PUT YOUR OWN HUGGINGFACE CREDENTIAL",
        lang_id_model_kind= "fasttext",
        lang_id_model_url= "facebook/fasttext-language-identification",
        run_locally= True,
        lang_id_content_column_name= "contents").transform()

In [ ]:
##### **** To explote the output from langId, run the code below
#table = pq.read_table('files-langId/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run Doc Quality

In [ ]:
%%capture
from dpk_doc_quality.ray.transform import DocQuality
DocQuality(input_folder='files-doc-chunk',
            output_folder= 'files-doc-quality',
            docq_text_lang = "en",
            run_locally= True,
            docq_doc_content_column ="contents").transform()

In [ ]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('files-doc-quality/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Run tokenization

In [ ]:
%%capture
from dpk_tokenization.ray.transform import Tokenization
Tokenization(input_folder= "files-doc-quality",
        output_folder= "files-tokenization",
        tkn_tokenizer=  "hf-internal-testing/llama-tokenizer",
        run_locally= True,
        tkn_chunk_size= 20_000).transform()

In [ ]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('files-tokenization/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()

Configure and run DocId

In [ ]:
from dpk_doc_id.transform_python import DocID
DocID(input_folder= "files-doc-chunk",
        output_folder= "files-doc-id",
        doc_id_doc_column= "contents",
        doc_id_hash_column= "document_id",
        doc_id_int_column= "int_id_column",
        doc_id_start_id= 5).transform()


In [ ]:
##### **** To explote the output from Doc Quality, run the code below
#table = pq.read_table('files-doc-id/arxiv_org_2408.09869v5.pdf_application.parquet')
#table.to_pandas()